## Transactions
#### 03.4 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2021-12-19


- Load credentials
- Interact with the blockchain
- Execute a payment from Python
- Add a note to a payment

## Setup
Starting with this chapter 3.4, the lines below will always automatically load ...
* The accounts MyAlgo, Alice, Bob, Charlie, Dina
* The Purestake credentials
* The functions in `algo_util.py`

In [1]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [2]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future.transaction import PaymentTxn, MultisigTransaction
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
import algosdk.error
import json

In [3]:
print(MyAlgo['public'])
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])
print(Dina['public'])

VL5UU2QXNKNEH7VISHZFU2ALXN5MOIDD3KXEYX2ADLCCIYN3MCOKRBATV4
VK6CCXY4IFHIJAVMRVS543LJQEQKOJO6YQ4DZNV3D2XJI4ETYBN5354EQU
CPUT3Z5CI3XOIZ4ARSGUFQD7V4YGYJW5BFAZMXX5YOV4KJCKI6MBCDY5XM
BY5K2AYO7R3G66ICY6SJ2JFVLRMIX677EAEEKDBTJZGP6Q4JVNZRDXDBKA
YXUPADIBAU7TOTY5MIOJIN6HDXS42SQCTMAPEUHU4GQCAKZQXJXXCQOJU4


### Check the accounts on the blockchain
- Go to https://algoexplorer.io and insert address
- Go to https://testnet.algoexplorer.io for the testnet
- Alternative indexers are https://algoscan.app and https://explorer.perawallet.app

In [4]:
# Create a link to directly access your MyAlgo account
print('https://algoexplorer.io/address/'+MyAlgo['public'])
print('https://testnet.algoexplorer.io/address/'+MyAlgo['public'])

https://algoexplorer.io/address/VL5UU2QXNKNEH7VISHZFU2ALXN5MOIDD3KXEYX2ADLCCIYN3MCOKRBATV4
https://testnet.algoexplorer.io/address/VL5UU2QXNKNEH7VISHZFU2ALXN5MOIDD3KXEYX2ADLCCIYN3MCOKRBATV4


### Fund with testnet Algos
- https://bank.testnet.algorand.network/
- https://testnet.algoexplorer.io/dispenser
- https://dispenser.testnet.aws.algodev.network
- Fund `MyAlgo`, `Alice` and `Bob`. How many test ALGOs did you get?

## Connecting to the Algorand Blockchain via API
+ We choose to connect via Purestake API
+ Requires Purestake token to authenticate
    - See 03.3_WSC_Credentials
- API **address** stored ...
    - For the testnet in `cred['algod_test']`
    - For the mainnet in `cred['algod_main']`
- API **crendentials** stored in `cred['purestake_token']`
  - Note: this is already the pair `{'X-Api-key': 'your token'}`
  - To obtain token alone use `cred['purestake_token']['X-Api-key']`
- See also: https://developer.algorand.org/tutorials/creating-python-transaction-purestake-api/  

In [5]:
algod_token   = ''                        # Only needed if we have our own server
algod_address = cred['algod_test']        # Or cred['algod_main'] 
purestake_token = cred['purestake_token'] # Authentication token pair {'X-Api-key': '(your token'}

# Initialize the algod client
algod_client = algod.AlgodClient(algod_token=algod_token, algod_address=algod_address, headers=purestake_token)

#### Test the connection
- Our first Python access of the blockchain
- What's the last block?
- Verify on https://testnet.algoexplorer.io
- Note that block count on testnet is larger (Why?)

In [6]:
algod_client.status()["last-round"]

26066624

### Obtain holdings

In [7]:
# Get holdings of testnet Algos
address = Alice["public"]
algod_client.account_info(address)["amount"]

56727106

In [8]:
# Holdings are in micro Algo ... convert
algo_precision = 1e6
algo_amount    = algod_client.account_info(address)["amount"]/algo_precision
print(f"Address {address} has {algo_amount} test algos")

Address VK6CCXY4IFHIJAVMRVS543LJQEQKOJO6YQ4DZNV3D2XJI4ETYBN5354EQU has 56.727106 test algos


#### Suggested parameters for a transaction (on the test network)

In [9]:
sp = algod_client.suggested_params()
print(json.dumps(vars(sp), indent=4))

{
    "first": 26066624,
    "last": 26067624,
    "gh": "SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=",
    "gen": "testnet-v1.0",
    "fee": 0,
    "flat_fee": false,
    "consensus_version": "https://github.com/algorandfoundation/specs/tree/433d8e9a7274b6fca703d91213e05c7e6a589e69",
    "min_fee": 1000
}


## A first payment transaction

#### Step 1: prepare and create unsigned transaction

In [10]:
# Parameters
sp       = algod_client.suggested_params()       # suggested params
amount   = 0.1
algo_precision = 1e6
amt_microalgo = int(amount * algo_precision)

# Create (unsigned) TX
txn = PaymentTxn(sender = Alice['public'],     # <--- From
                 sp = sp, 
                 receiver = Bob['public'],     # <---- To
                 amt = amt_microalgo           # <---- Amount in Micro-ALGOs
                )
print(txn)
print('')
print(txn.get_txid())

{'sender': 'VK6CCXY4IFHIJAVMRVS543LJQEQKOJO6YQ4DZNV3D2XJI4ETYBN5354EQU', 'fee': 1000, 'first_valid_round': 26066625, 'last_valid_round': 26067625, 'note': None, 'genesis_id': 'testnet-v1.0', 'genesis_hash': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=', 'group': None, 'lease': None, 'type': 'pay', 'rekey_to': None, 'receiver': 'CPUT3Z5CI3XOIZ4ARSGUFQD7V4YGYJW5BFAZMXX5YOV4KJCKI6MBCDY5XM', 'amt': 100000, 'close_remainder_to': None}

4MMDDBFUGQ3QLNZRKAAKPKLM6HXPMIFB6PHTPYSI6AIVLHMEA6GA


In [11]:
# Is it already on the blockchain? No ... not yet sent
print('https://testnet.algoexplorer.io/tx/'+txn.get_txid())

https://testnet.algoexplorer.io/tx/4MMDDBFUGQ3QLNZRKAAKPKLM6HXPMIFB6PHTPYSI6AIVLHMEA6GA


#### Step 2: sign

In [12]:
stxn = txn.sign(Alice['private'])                 # <---- Alice signs with private key

# The new stxn object consists of
print(stxn.transaction)
print('')
# and
print(stxn.signature)
print('')

{'sender': 'VK6CCXY4IFHIJAVMRVS543LJQEQKOJO6YQ4DZNV3D2XJI4ETYBN5354EQU', 'fee': 1000, 'first_valid_round': 26066625, 'last_valid_round': 26067625, 'note': None, 'genesis_id': 'testnet-v1.0', 'genesis_hash': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=', 'group': None, 'lease': None, 'type': 'pay', 'rekey_to': None, 'receiver': 'CPUT3Z5CI3XOIZ4ARSGUFQD7V4YGYJW5BFAZMXX5YOV4KJCKI6MBCDY5XM', 'amt': 100000, 'close_remainder_to': None}

xgcdxwJKCe2DX9+kymL5rZ/ec7vipKCggNX2tFzcUeTvPTiIEyu/x8ydIVaDKOz/eVm+lIo7pBk7NfdbFtwpAA==



In [13]:
# Transaction ID is the same, and still nothing on the blockchain
print('https://testnet.algoexplorer.io/tx/'+stxn.get_txid())

https://testnet.algoexplorer.io/tx/4MMDDBFUGQ3QLNZRKAAKPKLM6HXPMIFB6PHTPYSI6AIVLHMEA6GA


#### Step 3: send

In [14]:
txid = algod_client.send_transaction(stxn)
print("Send transaction with txID: "+txid)

# The freshly submitted transaction on the blockchain
txinfo = algod_client.pending_transaction_info(txid)
print(txinfo)

Send transaction with txID: 4MMDDBFUGQ3QLNZRKAAKPKLM6HXPMIFB6PHTPYSI6AIVLHMEA6GA
{'pool-error': '', 'txn': {'sig': 'xgcdxwJKCe2DX9+kymL5rZ/ec7vipKCggNX2tFzcUeTvPTiIEyu/x8ydIVaDKOz/eVm+lIo7pBk7NfdbFtwpAA==', 'txn': {'amt': 100000, 'fee': 1000, 'fv': 26066625, 'gen': 'testnet-v1.0', 'gh': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=', 'lv': 26067625, 'rcv': 'CPUT3Z5CI3XOIZ4ARSGUFQD7V4YGYJW5BFAZMXX5YOV4KJCKI6MBCDY5XM', 'snd': 'VK6CCXY4IFHIJAVMRVS543LJQEQKOJO6YQ4DZNV3D2XJI4ETYBN5354EQU', 'type': 'pay'}}}


#### Step 4: Wait for confirmation

In [15]:
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  26066626.
Waiting for round 26066626 to finish.
Waiting for round 26066627 to finish.
Transaction 4MMDDBFUGQ3QLNZRKAAKPKLM6HXPMIFB6PHTPYSI6AIVLHMEA6GA confirmed in round 26066628.


In [16]:
# Note that txinfo has now a 'confirmed-round'
print(txinfo)
print('https://testnet.algoexplorer.io/tx/'+txid)

{'confirmed-round': 26066628, 'pool-error': '', 'txn': {'sig': 'xgcdxwJKCe2DX9+kymL5rZ/ec7vipKCggNX2tFzcUeTvPTiIEyu/x8ydIVaDKOz/eVm+lIo7pBk7NfdbFtwpAA==', 'txn': {'amt': 100000, 'fee': 1000, 'fv': 26066625, 'gen': 'testnet-v1.0', 'gh': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=', 'lv': 26067625, 'rcv': 'CPUT3Z5CI3XOIZ4ARSGUFQD7V4YGYJW5BFAZMXX5YOV4KJCKI6MBCDY5XM', 'snd': 'VK6CCXY4IFHIJAVMRVS543LJQEQKOJO6YQ4DZNV3D2XJI4ETYBN5354EQU', 'type': 'pay'}}}
https://testnet.algoexplorer.io/tx/4MMDDBFUGQ3QLNZRKAAKPKLM6HXPMIFB6PHTPYSI6AIVLHMEA6GA


## Add a note to a transaction

In [17]:
# Step 1a: Prepare
sp       = algod_client.suggested_params()       # suggested params

amount    = 0.1
algo_precision = 1e6
amt_microalgo = int(amount * algo_precision)

# Step 1b: The note
note_txt  = "Paying back for last dinner"
note_byte = note_txt.encode()

In [18]:
# Step 1c: create (unsigned) TX
txn = PaymentTxn(sender=Alice['public'],
                 sp=sp, 
                 receiver = Bob['public'],
                 amt=amt_microalgo, 
                 note=note_byte
                 )
print(txn)

{'sender': 'VK6CCXY4IFHIJAVMRVS543LJQEQKOJO6YQ4DZNV3D2XJI4ETYBN5354EQU', 'fee': 1000, 'first_valid_round': 26066628, 'last_valid_round': 26067628, 'note': b'Paying back for last dinner', 'genesis_id': 'testnet-v1.0', 'genesis_hash': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=', 'group': None, 'lease': None, 'type': 'pay', 'rekey_to': None, 'receiver': 'CPUT3Z5CI3XOIZ4ARSGUFQD7V4YGYJW5BFAZMXX5YOV4KJCKI6MBCDY5XM', 'amt': 100000, 'close_remainder_to': None}


In [19]:
# Step 2+3: sign and send TX
stxn = txn.sign(Alice['private'])
txid = algod_client.send_transaction(stxn)
print("Send transaction with txID: {}".format(txid))

Send transaction with txID: MCB6AAQ6BLMNA6HE7HXCFR5TNNSZCFEQLOPZ7OYNTIXRSWASFUOQ


In [20]:
# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  26066627.
Waiting for round 26066627 to finish.
Waiting for round 26066628 to finish.
Waiting for round 26066629 to finish.
Transaction MCB6AAQ6BLMNA6HE7HXCFR5TNNSZCFEQLOPZ7OYNTIXRSWASFUOQ confirmed in round 26066630.


In [21]:
print("https://testnet.algoexplorer.io/tx/"+txid)

https://testnet.algoexplorer.io/tx/MCB6AAQ6BLMNA6HE7HXCFR5TNNSZCFEQLOPZ7OYNTIXRSWASFUOQ


In [22]:
print(txinfo)

{'confirmed-round': 26066630, 'pool-error': '', 'txn': {'sig': 'DgbrVgZ6I1+dT+vARBPFxJzEvghilBygR56E63emyivKLDkeGZkEzIpCiwp0HZeeUY9seWKhI8biqfP0E1AxCQ==', 'txn': {'amt': 100000, 'fee': 1000, 'fv': 26066628, 'gen': 'testnet-v1.0', 'gh': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=', 'lv': 26067628, 'note': 'UGF5aW5nIGJhY2sgZm9yIGxhc3QgZGlubmVy', 'rcv': 'CPUT3Z5CI3XOIZ4ARSGUFQD7V4YGYJW5BFAZMXX5YOV4KJCKI6MBCDY5XM', 'snd': 'VK6CCXY4IFHIJAVMRVS543LJQEQKOJO6YQ4DZNV3D2XJI4ETYBN5354EQU', 'type': 'pay'}}}


In [23]:
# Convert message in txinfo to Python dict
import base64
note_base64 = txinfo['txn']['txn']['note']
print(note_base64)
note_byte   = base64.b64decode(message_base64)
print(note_byte)
note_txt   = message_byte.decode()
print(note_txt)

UGF5aW5nIGJhY2sgZm9yIGxhc3QgZGlubmVy


NameError: name 'message_base64' is not defined

## Useful functions
These function `wait_for_confirmation` is an ufficial Algorand function.<br>
Below is the source code.

In [ ]:
def wait_for_confirmation(client, txid):
    # client = algosdk client
    # txid = transaction ID, for example from send_payment()

    txinfo = client.pending_transaction_info(txid)       # obtain transaction information
    current_round = algod_client.status()["last-round"]        # obtain last round number
    print("Current round is  {}.".format(current_round))
    
    # Wait for confirmation
    while ( txinfo.get('confirmed-round') is None ):            # condition for waiting = 'confirmed-round' is empty
        print("Waiting for round {} to finish.".format(current_round))
        algod_client.status_after_block(current_round)             # this wait for the round to finish
        txinfo = algod_client.pending_transaction_info(txid)    # update transaction information
        current_round += 1

    print("Transaction {} confirmed in round {}.".format(txid, txinfo.get('confirmed-round')))
    return txinfo

## Also useful functions
These functions are much more convenient:
- `note_encode` encodes a note from a Pyhon dict
- `note_decode` decodes a note into a Pyhon dict

In [ ]:
def note_encode(note_dict):
    # note dict ... a Python dictionary
    note_json = json.dumps(note_dict)
    note_byte = note_json.encode()     
    return(note_byte)

def note_decode(note_64):
    # note64 =  note in base64 endocing
    # returns a Python dict
    import base64
    message_base64 = txinfo['txn']['txn']['note']
    message_byte   = base64.b64decode(message_base64)
    message_json   = message_byte.decode()
    message_dict   = json.loads( message_json )
    return(message_dict)

### Exercise
* Send 0.8 ALGO from Dina to Charlie with a thank you note

In [24]:
# Your Python code goes here

## Things that do not and will not work
Let's produce some error messages. Following are a few things that don't work

In [25]:
# Need to import this to be able to read error messages
import sys, algosdk.error

### Overspending
Alice sends more than she owns

In [26]:
# Step 1: prepare
sp       = algod_client.suggested_params()
algo_precision = 1e6
sender   = Alice['public']
receiver = Bob['public']
amount   = 100                       # <----------------- way too much!
amount_microalgo = int(amount * algo_precision)

# Step 2: create unsigned TX
unsigned_txn = PaymentTxn(sender, sp, receiver, amount_microalgo)

# Step 3a: Sign
signed_txn = unsigned_txn.sign(Alice['private'])

In [27]:
# Step 3b: Send
txid = algod_client.send_transaction(signed_txn)

AlgodHTTPError: TransactionPool.Remember: transaction E3U5ZHGAH3XPWKRXWA7HY7KCHKOZ2E6SUM6OEW3RFGEU4PL5OD7Q: overspend (account VK6CCXY4IFHIJAVMRVS543LJQEQKOJO6YQ4DZNV3D2XJI4ETYBN5354EQU, data {AccountBaseData:{Status:Offline MicroAlgos:{Raw:56524106} RewardsBase:27521 RewardedMicroAlgos:{Raw:0} AuthAddr:AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAY5HFKQ TotalAppSchema:{_struct:{} NumUint:23 NumByteSlice:8} TotalExtraAppPages:0 TotalAppParams:0 TotalAppLocalStates:11 TotalAssetParams:32 TotalAssets:48 TotalBoxes:0 TotalBoxBytes:0} VotingData:{VoteID:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] SelectionID:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] StateProofID:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] VoteFirstValid:0 VoteLastValid:0 VoteKeyDilution:0}}, tried to spend {100000000})

#### Can we *catch the error* and get a better structured error message?

In [28]:
# Step 1: prepare
sp       = algod_client.suggested_params()
algo_precision = 1e6
sender   = Alice['public']
receiver = Bob['public']
amount   = 100                       # <----------------- way too much!
amount_microalgo = int(amount * algo_precision)

# Step 2: create unsigned TX
unsigned_txn = PaymentTxn(sender, sp, receiver, amount_microalgo)

# Step 3a: Sign
signed_txn = unsigned_txn.sign(Alice['private'])

In [29]:
# Step 3b: Send
try:
    txid = algod_client.send_transaction(signed_txn)
except algosdk.error.AlgodHTTPError as err:
    print(err)                                   # print entire error message
    if ("overspend" in str(err)):                # check for specific type of error
        print("Overspend error")         
    txid = None

TransactionPool.Remember: transaction E3U5ZHGAH3XPWKRXWA7HY7KCHKOZ2E6SUM6OEW3RFGEU4PL5OD7Q: overspend (account VK6CCXY4IFHIJAVMRVS543LJQEQKOJO6YQ4DZNV3D2XJI4ETYBN5354EQU, data {AccountBaseData:{Status:Offline MicroAlgos:{Raw:56524106} RewardsBase:27521 RewardedMicroAlgos:{Raw:0} AuthAddr:AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAY5HFKQ TotalAppSchema:{_struct:{} NumUint:23 NumByteSlice:8} TotalExtraAppPages:0 TotalAppParams:0 TotalAppLocalStates:11 TotalAssetParams:32 TotalAssets:48 TotalBoxes:0 TotalBoxBytes:0} VotingData:{VoteID:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] SelectionID:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] StateProofID:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] VoteFirstValid:0 VoteLastValid:0 VoteKeyDilution:0}}, tried to spend {100000000})
Overspend error


#### What happens if we wait for the failed transaction to complete?

In [30]:
# We fail at the first command
try:
    txinfo = algod_client.pending_transaction_info(txid)       # obtain transaction information
    print(txinfo)
except TypeError as err:                                       # obtain error message
    # print entire error message (rather cryptic!)
    print(err)
    # check for specific type of error
    print("txid is empty")

can only concatenate str (not "NoneType") to str
txid is empty


### Wrong signature
Bob tries to sign a transaction from Alice to Bob

In [31]:
# Step 1: prepare
sp       = algod_client.suggested_params()
algo_precision = 1e6
sender   = Alice['public']
receiver = Bob['public']
amount   = 0.1
amount_microalgo = int(amount * algo_precision)

# Step 2: create unsigned TX
unsigned_txn = PaymentTxn(sender, sp, receiver, amount_microalgo)

# Step 3a: Sign
signed_txn = unsigned_txn.sign(Bob['private'])                        # <----------------- wrong person signs!

try:
    txid = algod_client.send_transaction(signed_txn)
except algosdk.error.AlgodHTTPError as err:
    # print entire error message
    print(err)
    if ("should have been authorized" in str(err)):                # check for specific type of error
        print("Wrong signature error")         
    txid = None

TransactionPool.Remember: transaction 3GWX7RSVW6Y7NONEX3TXYWSAMNTURMD6ZIDXFI5XS3QEQU2UFPZQ: should have been authorized by VK6CCXY4IFHIJAVMRVS543LJQEQKOJO6YQ4DZNV3D2XJI4ETYBN5354EQU but was actually authorized by CPUT3Z5CI3XOIZ4ARSGUFQD7V4YGYJW5BFAZMXX5YOV4KJCKI6MBCDY5XM
Wrong signature error


### Sending the *indentical* transaction twice
* "Identical" means same ...
    * Sender
    * Recipient
    * Ammount
    * (Suggested) parameters (including first/last round)

In [32]:
# Step 1: prepare
sp       = algod_client.suggested_params()
algo_precision = 1e6
sender   = Alice['public']
receiver = Bob['public']
amount   = 0.1
amount_microalgo = int(amount * algo_precision)

In [33]:
# Step 2: create unsigned TX
unsigned_txn = PaymentTxn(sender, sp, receiver, amount_microalgo)

# Step 3: Sign and send
signed_txn = unsigned_txn.sign(Alice['private'])
try:
    txid = algod_client.send_transaction(signed_txn)
    print("Submitted with txID: {}".format(txid))
except algosdk.error.AlgodHTTPError as err:
    # print entire error message
    print(err)
    if ("transaction already in ledger" in str(err)):                # check for specific type of error
        print("Identical transaction {} has been submitted twice.".format(signed_txn.get_txid()))         
    txid = None    # check for specific type of error

Submitted with txID: FF2WBF2MBHB5ZXF3IHFACMQ5L62IUN6M7NVB5P2SKSBHX6XVUZMQ


**REPEAT** only step 2-3 $\rightarrow$ error message<br>
**REPEAT** only step 1-3 $\rightarrow$ no error <br>

#### See how the sp change
* Re-run this after 2-3 seconds

In [34]:
sp = algod_client.suggested_params()
print(json.dumps(vars(sp), indent=4))

{
    "first": 26066646,
    "last": 26067646,
    "gh": "SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=",
    "gen": "testnet-v1.0",
    "fee": 0,
    "flat_fee": false,
    "consensus_version": "https://github.com/algorandfoundation/specs/tree/433d8e9a7274b6fca703d91213e05c7e6a589e69",
    "min_fee": 1000
}
